In [29]:
import torch
#from onnx_tf.backend import prepare
import onnx
import tensorflow as tf

In [30]:
# loaded = torch.jit.load('7x7/dualnet7x7_16conv.pt')
# loaded = torch.jit.load('5x5/best.pt')
# loaded = torch.jit.load('src/dualnet5x5_v2.pt')
# loaded = torch.jit.load('src/dualnet5x5_se.pt')
loaded = torch.jit.load('src/dualnet7x7_bn.pt')
# print(loaded)

In [31]:
loaded.eval()
bisze = 7
dummy_input = torch.randn((8, 12, bisze, bisze)).to("cuda:0")
torch.onnx.export(loaded, dummy_input, "dualnet.onnx", do_constant_folding=True, export_params=True, verbose=True)

graph(%s.1 : Float(8, 12, 7, 7, strides=[588, 49, 7, 1], requires_grad=0, device=cuda:0),
      %conv1.bias : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %conv1.weight : Float(48, 12, 3, 3, strides=[108, 9, 3, 1], requires_grad=0, device=cuda:0),
      %bn1.running_var : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn1.running_mean : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn1.bias : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn1.weight : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %conv2.bias : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %conv2.weight : Float(48, 48, 3, 3, strides=[432, 9, 3, 1], requires_grad=0, device=cuda:0),
      %bn2.running_var : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn2.running_mean : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn2.bias : Float(48, strides=[1], requires_grad=0, device=cuda:0),
      %bn2

In [17]:
# from onnx_tf.backend import prepare

# # tf.autograph.experimental.do_not_convert
# onnx_model = onnx.load('dualnet.onnx')
# tf_rep = prepare(onnx_model, device='gpu')
# tf_rep.export_graph('model.pb')

In [18]:
# !onnx-tf convert -i dualnet.onnx -o onnxout
# !tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' --saved_model_tags=serve onnxout js_model --quantize_uint8

In [32]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = 'dualnet.onnx'
model_quant = 'quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

In [12]:
# !chcp 65001
# !cd train
# !saved_model_cli show --dir ./model.pb --all

Active code page: 65001


The system cannot find the path specified.
'saved_model_cli' is not recognized as an internal or external command,
operable program or batch file.


In [14]:

# !tensorflowjs_converter --input_format=tf_saved_model --output_node_names='output_0' --saved_model_tags=serve model.pb js_model --quantize_uint8

Traceback (most recent call last):
  File "C:\Users\sen\anaconda3\envs\az\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\sen\anaconda3\envs\az\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\sen\anaconda3\envs\az\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "C:\Users\sen\anaconda3\envs\az\lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "C:\Users\sen\anaconda3\envs\az\lib\site-packages\tensorflowjs\converters\__init__.py", line 21, in <module>
    from tensorflowjs.converters.converter import convert
  File "C:\Users\sen\anaconda3\envs\az\lib\site-packages\tensorflowjs\converters\converter.py", line 28, in <module>
    import h5py
ModuleNotFoundError: No module named 'h5py'


In [1]:
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

c:\Users\sen\anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\sen\anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\sen\anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\sen\anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


KeyboardInterrupt: 

In [ ]:
new_model = tf.saved_model.load('onnxout.pb')

In [ ]:
import onnx
from onnx2keras import onnx_to_keras

# Load ONNX model
onnx_model = onnx.load('dualnet.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(onnx_model, ['x.1'])

TypeError: Expected `name` argument to be a string, but got: ['input.3', '116', '117', 'batch_norm_dead_output-128', 'batch_norm_dead_output-129']